In [ ]:
# In this notebook we will be using a Custom CNN model to train for only one class but with a GPU

import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

print(device)

Mounted at /content/drive
cuda


In [ ]:
"Let us first divide the test and train data into proper format"

import os
import shutil
import random

base_dir = "animals"

animal_dir = "//content//drive//My Drive//DeepLearningCollab//Animal"
os.makedirs(animal_dir , exist_ok=True)

# Now we go into the directry containing the animals


source_dir = "//content//drive//My Drive//DeepLearningCollab//animals"

#Destination Directories

train_dir = os.path.join(animal_dir , 'train')
test_dir = os.path.join(animal_dir , 'test')

"If train and test directories do not exist"

os.makedirs(train_dir , exist_ok=True)
os.makedirs(test_dir , exist_ok=True)

"Now  we can list all the directories in source_dir"

animal_directories_list = os.listdir(source_dir)

"Now we will loop through the directories"

for animal in animal_directories_list:
    animal_D = os.path.join(source_dir , animal)

    # Get a list of all the image files in the directory animal_D
    images = [i for i in os.listdir(animal_D) if os.path.isfile(os.path.join(animal_D , i))]

    train_test_spllit = 0.8

    num_train = int(len(images) * train_test_spllit)

    random.shuffle(images) # Shuffle the images randomly
    train_images = images[:num_train]

    test_images = images[num_train:]


    # Now we have already created test and train directory but here we have to also create test and train for each of the animals

    train_dir_animal = os.path.join(train_dir , animal)
    test_dir_animal = os.path.join(test_dir , animal)

    os.makedirs(train_dir_animal , exist_ok=True)
    os.makedirs(test_dir_animal , exist_ok=True)

    "Finally we have to move all the images from source directory to destination directory using shuttle"
    for img in train_images:
        shutil.move(os.path.join(animal_D , img) , os.path.join(train_dir_animal , img))
    for img in test_images:
        shutil.move(os.path.join(animal_D , img) , os.path.join(test_dir_animal , img))

In [ ]:
from pathlib import Path


train_path = os.path.join(animal_dir , "train")
test_path = os.path.join(animal_dir , "test")

animal_dir = Path(animal_dir)



# Using This we have also found out the class names and now we can convert them into Dict format

from typing import Tuple , Dict , List
train_path = Path(train_path)
test_path = Path(test_path)

def find_classes(directory : str) -> Tuple[List[str] , Dict[str,int]] :

    "Get the class_names first"

    classes = sorted(
        entry.name for entry in list(os.scandir(directory)) if entry.is_dir()
    )

    # Get the case to handle any kind of error

    if not classes:
        raise FileNotFoundError(f"Could not find any classes in {directory}")

    classes_to_idx = {cls_name : i for i , cls_name in enumerate(classes)}

    return classes , classes_to_idx

In [ ]:
"Now we will enter into the part where we will be transforming the image and creating Datasets and DataLoaders"
from torch.utils.data import DataLoader , TensorDataset , Dataset
from torchvision import transforms
import random
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt

"Lets write the transform for training Data and Testing Data"

transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p = 0.5), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

"""
Creating The Custom Dataset For One Vs Rest Classification:

1) In the custom dataset we must take in decide which will be our target class and which will be the other classes.
2) The target class will have the label as 1 and the other classes will have label as 0

"""
class CustomDataset(Dataset):

    def __init__(self , target_class_index , target_directory : str , transform = None):
        super().__init__()
        self.label = []
        self.img = []
        self.transform = transform

        # Here we can find the classes using find classes method

        classes , _ = find_classes(target_directory)
        target_class_name = classes[target_class_index]

        for Class in classes:

            if(Class == target_class_name):
              limit = 1
            else:
              limit = 0.1

            directory = os.path.join(target_directory , Class)
            for l , name in enumerate(os.listdir(directory)):
                if(l>=limit*len(os.listdir(directory))):
                    break
                final_path = os.path.join(directory , name)
                self.img.append(final_path)

                if(Class == target_class_name):
                    self.label.append(1)
                else:
                    self.label.append(0)

    def __len__(self):
        return len(self.img)

    def load_image(self, index: int) -> Image.Image:
        image_path = self.img[index]
        img = Image.open(image_path)
        return img

    def __getitem__(self, idx: int):
        image = Image.open(self.img[idx])


        Label = self.label[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(Label, dtype=torch.long)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from sklearn.model_selection import KFold

# Define your CustomDataset class and other necessary imports
# Assuming you have already defined CustomDataset and other imports



class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))

        x = self.relu(self.batch_norm2(self.conv2(x)))

        x = self.conv3(x)
        x = self.batch_norm3(x)

        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)

        return x

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()


        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
      identity = x.clone()

      x = self.relu(self.batch_norm2(self.conv1(x)))
      x = self.batch_norm2(self.conv2(x))

      if self.i_downsample is not None:
          identity = self.i_downsample(identity)
      print(x.shape)
      print(identity.shape)
      x += identity
      x = self.relu(x)
      return x




class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)

        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)

    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)

        return x

    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []

        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )

        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion

        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))

        return nn.Sequential(*layers)

def ResNet50(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

def ResNet101(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,23,3], num_classes, channels)

def ResNet152(num_classes, channels=3):
    return ResNet(Bottleneck, [3,8,36,3], num_classes, channels)
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, train_test_split

class_list , _ = find_classes(train_path)

# Define Stratified K-Fold cross-validation
skf = KFold(n_splits=3, shuffle=True)

for i in range(90):
  print(class_list[i])

  # Make the lists for the confusion Matrix
  true_labels = []
  predicted_labels = []

  # Load Custom Training Dataset And Testing Dataset

  train_dataset = CustomDataset(i , train_path , transform = transforms_train)
  test_dataset = CustomDataset(i , test_path , transform = transforms_test)

  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=24, shuffle=True, num_workers=2)

  #### Train model

  from sklearn.model_selection import KFold

  train_loss=[]
  train_accuracy=[]
  test_loss=[]
  test_accuracy=[]


  # Extract indices
  indices = list(range(len(train_dataset)))
  # Define Stratified K-Fold cross-validation
  skf = KFold(n_splits=3, shuffle=True)

  # Iterate over folds
  for fold, (train_index, val_index) in enumerate(skf.split(indices, [train_dataset.label[idx] for idx in indices])):

      val_true_labels = []
      val_predicted_labels = []

      print("FOLD : " , fold)
      train_sampler = torch.utils.data.SubsetRandomSampler(train_index)
      val_sampler = torch.utils.data.SubsetRandomSampler(val_index)

      # Create data loaders for training and validation
      train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=24, sampler=train_sampler, num_workers=4)
      val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=24, sampler=val_sampler, num_workers=4)

      model = ResNet50(2)

      # Since Resnet50 has 1000 out_features we will need to change it because our model has 1000 features.

      num_features = model.fc.in_features
      # Add a fully-connected layer for classification
      model.fc = nn.Linear(num_features, 2)
      model = model.to(device)

      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)




      num_epochs = 25
      for epoch in range(num_epochs):
        epoch_true_labels = []
        epoch_predicted_labels = []
        print("Epoch {} running".format(epoch)) #(printing message)
        model.train()
        running_loss = 0
        running_corrects = 0
        total_train = 0

        # Now Load A Batch Of Images

        for i , (inputs , labels) in enumerate(train_loader):
          inputs = inputs.to(device)
          labels = labels.to(device)

          # Forward Inputs and Get Outputs
          optimizer.zero_grad()
          outputs = model.forward(inputs)

          _ , preds = torch.max(outputs , 1)

          loss = criterion(outputs , labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          total_train += labels.size(0)

          running_corrects += torch.sum(preds == labels.data).item()

        epoch_loss = running_loss / len(train_sampler)
        epoch_acc = running_corrects / total_train *100

        # Append Result

        train_loss.append(epoch_loss)
        train_accuracy.append(epoch_acc)

        # Print Progress

        print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch+1, epoch_loss, epoch_acc))

        # Testing Part

        model.eval()
        with torch.no_grad():
          running_loss = 0
          running_corrects = 0
          total_val = 0

          for i , (inputs , labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model.forward(inputs)
            _ , preds = torch.max(outputs , 1)
            loss = criterion(outputs , labels)

            running_loss += loss.item()
            total_val += labels.size(0)
            running_corrects += torch.sum(preds == labels.data).item()
            val_true_labels.extend(labels.cpu().numpy())
            val_predicted_labels.extend(preds.cpu().numpy())
            epoch_true_labels.extend(labels.cpu().numpy())
            epoch_predicted_labels.extend(preds.cpu().numpy())



          epoch_loss = running_loss / len(val_sampler)
          epoch_acc = running_corrects / total_val *100

          # Append result

          test_loss.append(epoch_loss)
          test_accuracy.append(epoch_acc)

          # Print progress
          print('[Test #{}] Loss: {:.4f} Acc: {:.4f}% '.format(epoch+1, epoch_loss, epoch_acc))

          # Print Confusion Matrix After every epoch

      print('Confusion Matrix on the validation set Fold :' , fold)
      print(confusion_matrix(val_true_labels, val_predicted_labels))

  plt.figure(figsize=(10, 7))
  train_accuracy_tensor = torch.tensor(train_accuracy).cpu()
  test_accuracy_tensor = torch.tensor(test_accuracy).cpu()

  plt.plot(train_accuracy_tensor, color='green', label='train accuracy')
  plt.plot(test_accuracy_tensor, color='blue', label='validation accuracy')
  plt.xlabel('epochs')
  plt.ylabel('accuracy')
  plt.legend()
  plt.show()

  with torch.no_grad():
    running_corrects = 0
    total_test = 0

    for i , (inputs , labels) in enumerate(test_loader):
      inputs = inputs.to(device)
      labels = labels.to(device)
      outputs = model.forward(inputs)
      _ , preds = torch.max(outputs , 1)
      running_corrects += torch.sum(preds == labels.data).item()
      total_test += labels.size(0)
      true_labels.extend(labels.cpu().numpy())
      predicted_labels.extend(preds.cpu().numpy())

    epoch_acc = running_corrects / total_test *100

    print("Test_Accuracy :" , epoch_acc)

    print('Confusion Matrix on the test set')
    print(confusion_matrix(true_labels, predicted_labels))
















Output hidden; open in https://colab.research.google.com to view.